In [1]:
%pip install pandas matplotlib openpyxl
import pandas as pd
import matplotlib.pyplot as plt
import re


import datetime
import os

    

Note: you may need to restart the kernel to use updated packages.


In [3]:


#用pd读取input下的几个excel文件
df_cient_summary = pd.read_excel('./input/生意参谋_客户概况.xlsx').iloc[:,:8]
df_follows_views = pd.read_excel('./input/生意参谋-客户关注分析-核心指标.xlsx').iloc[:,:5]
df_follows_contents_new = pd.read_excel('./input/生意参谋-客户关注分析-上新内容.xlsx').iloc[:,:2]
df_follows_contents_list = pd.read_excel('./input/生意参谋-客户关注分析-清单内容.xlsx').iloc[:,:2]
df_follows_contents = pd.read_excel('./input/生意参谋-内容管理-关注.xlsx').iloc[:,:2]
print("客户概况")
print(df_cient_summary.columns)
print("生意参谋-客户关注分析-核心指标")
print(df_follows_views.columns)
print("生意参谋-客户关注分析-上新内容")
print(df_follows_contents_new.columns)
print("生意参谋-客户关注分析-清单内容")
print(df_follows_contents_list.columns)
print("生意参谋-内容管理-关注")
print(df_follows_contents.columns)

客户概况
Index(['日期', '店铺客户数', '新访', '新访成交', '未购回访', '未购回访成交', '老客回访', '老客回访成交'], dtype='object')
生意参谋-客户关注分析-核心指标
Index(['日期', '曝光人数', '浏览人数', '互动人数', '进店人数'], dtype='object')
生意参谋-客户关注分析-上新内容
Index(['日期', '上新内容列表按曝光人数排序'], dtype='object')
生意参谋-客户关注分析-清单内容
Index(['日期', '清单内容列表按曝光人数排序'], dtype='object')
生意参谋-内容管理-关注
Index(['关注内容列表', '页码'], dtype='object')


In [4]:
#把新抓的数据合并，并导出新表，把老表用一个old目录保存
df_cient_summary_TEMP = pd.read_excel('./input/temp/生意参谋_客户概况.xlsx').iloc[:,:8]
df_follows_views_TEMP = pd.read_excel('./input/temp/生意参谋-客户关注分析-核心指标.xlsx').iloc[:,:5]
df_follows_contents_new_TEMP = pd.read_excel('./input/temp/生意参谋-客户关注分析-上新内容.xlsx').iloc[:,:2]
df_follows_contents_list_TEMP = pd.read_excel('./input/temp/生意参谋-客户关注分析-清单内容.xlsx').iloc[:,:2]
df_follows_contents_TEMP = pd.read_excel('./input/temp/生意参谋-内容管理-关注.xlsx').iloc[:,:2]


In [5]:
#合并历史数据
df_cient_summary = pd.concat([df_cient_summary_TEMP, df_cient_summary], ignore_index=True).drop_duplicates(subset='日期', keep='first').reset_index(drop=True)
df_follows_views = pd.concat([df_follows_views_TEMP, df_follows_views], ignore_index=True).drop_duplicates(subset='日期', keep='first').reset_index(drop=True)
df_follows_contents_new = pd.concat([df_follows_contents_new_TEMP, df_follows_contents_new], ignore_index=True).drop_duplicates(subset=['日期','上新内容列表按曝光人数排序'], keep='first').reset_index(drop=True)
df_follows_contents_list = pd.concat([df_follows_contents_list_TEMP, df_follows_contents_list], ignore_index=True).drop_duplicates(subset=['日期','清单内容列表按曝光人数排序'], keep='first').reset_index(drop=True)
df_follows_contents = pd.concat([df_follows_contents_TEMP, df_follows_contents], ignore_index=True).drop_duplicates(subset='关注内容列表', keep='first').reset_index(drop=True)



In [6]:
###下面代码移动旧数据，通过判断文件夹是否为空，来避免重复移动覆盖旧数据######################
import shutil

today = datetime.date.today()
folder_name = today.strftime("%Y%m%d")

input_dir = './input/old/'
folder_path = os.path.join(input_dir, folder_name)

if not os.path.exists(folder_path):
    os.makedirs(folder_path)

#把旧的文件移动到这个文件夹，如果有相同名字就直接覆盖
folder_path = os.path.join(input_dir, folder_name)

files_to_move = [
    './input/生意参谋_客户概况.xlsx',
    './input/生意参谋-客户关注分析-核心指标.xlsx',
    './input/生意参谋-客户关注分析-上新内容.xlsx',
    './input/生意参谋-客户关注分析-清单内容.xlsx',
    './input/生意参谋-内容管理-关注.xlsx'
]

if not os.listdir(folder_path):
    for file in files_to_move:
        try:
            shutil.move(file, folder_path)
        except PermissionError as e:
            print(f"PermissionError: Could not move {file}. It might be open in another program. Error: {e}")
else:
    print("Folder is not empty, files will not be moved.")

In [7]:
#再导出新的文件到input目录
df_cient_summary.to_excel('./input/生意参谋_客户概况.xlsx', index=False)
df_follows_views.to_excel('./input/生意参谋-客户关注分析-核心指标.xlsx', index=False)
df_follows_contents_new.to_excel('./input/生意参谋-客户关注分析-上新内容.xlsx', index=False)
df_follows_contents_list.to_excel('./input/生意参谋-客户关注分析-清单内容.xlsx', index=False)
df_follows_contents.to_excel('./input/生意参谋-内容管理-关注.xlsx', index=False)

## 关注信息流内容 - 数据清理

In [44]:
# 删除哪些没有日期信息数据
df_follows_contents = df_follows_contents[df_follows_contents['关注内容列表'].str.contains(r'\d{4}-\d{2}-\d{2}', regex=True, na=False)]

#把 关注内容列表， 按照空行拆分为一个list，记住是空行
df_follows_contents['关注内容列表2'] = df_follows_contents['关注内容列表'].str.split('\n\t\n')

# 将拆分后的列表转换为 DataFrame
split_columns = df_follows_contents['关注内容列表2'].apply(pd.Series)


In [45]:
#清理内容列表
#定义一个函数来提取数据
# def extract_data(text):
#     # 拆分文本为几个模块
#     sections = [section for section in text.split('\n\n')]
#     # 提取每个模块的标题和内容
#     title = sections[0].split('：')[1]

#     return {
#         "内容标题"：title,
# }


# # 示例数据
# data = [
#     "高级感衬衫！ | 衬衫怎么搭都好看，让你成为穿搭高手！\nID: 24255682137310\n多品上新\n\n2025-04-11\n20:11:12\n\n猜你喜欢已采纳\n\n已发布\n\n查看数据下线评论管理"
#     # 可以继续添加更多的内容
# ]

# # 创建DataFrame
# df = pd.DataFrame(data, columns=["内容"])

# 定义一个函数来提取信息
def extract_info(text):
    # 拆分文本，按空行分割
    sections = [section.strip() for section in text.split('\n\t\n')]
    #print(sections)
    # 提取各个模块内容
    title = sections[0].strip().split('\n')[0]  # 内容标题, 第一行
    content_id = re.search(r'ID:\s*(\d+)', sections[0]).group(1)  # id
    content_type = sections[0].strip().split('\n')[-1]  # 内容类型，最后一行
    publish_date_match = re.search(r'\d{4}-\d{2}-\d{2}', sections[1])  # 发布日期。 不能用位置判断，因为有字符“定时”
    if publish_date_match:
        publish_date = publish_date_match.group()  # 获取匹配的日期字符串
    else:
        publish_date = None  # 如果没有匹配，设置为 None
    
    publish_time = sections[1].strip().split('\n')[1]  # 发布时间
    adopted = sections[2].strip()  # 是否被采纳
    published = sections[3].strip()  # 是否已发布
    comments = sections[4].strip()  # 查看评论

    # 返回结果
    return {
        "内容标题": title,
        "id": content_id,
        "内容类型": content_type,
        "发布日期": publish_date,
        "发布时间": publish_time,
        "是否被采纳": adopted,
        "是否已发布": published,
        "查看评论": comments
    }

# 应用函数，提取所有行的信息
extracted_data = df_follows_contents["关注内容列表"].apply(extract_info)
print(len(extracted_data))

# 将提取到的数据转换为新的DataFrame
extracted_df = pd.DataFrame(extracted_data.tolist(),dtype=str)
df_follows_contents2 = extracted_df



1077


In [46]:

# 统计每个日期的行数； 统计每个日期和内容类型的行数

# 统计每个日期的行数
df_counts = df_follows_contents2.groupby('发布日期').size().reset_index(name='行数')
# 统计每个日期和内容类型的行数，日期列保持唯一值，每种数据类型作为1列，值就是行数

df_counts2 = df_follows_contents2.groupby(['发布日期', '内容类型']).size().reset_index(name='行数')
#df_counts2.to_excel('TEST.xlsx')


# 透视表：行是“发布日期”，列是“内容类型”，值是“行数”
pivot_df = df_counts2.pivot_table(index='发布日期', 
                          columns='内容类型', 
                          values='行数', 
                          fill_value=0)
#pivot_df.to_excel('TEST.xlsx')
#print(pivot_df)


# #把客户关注分析表、df_counts，df_counts2合并， 以df_counts的‘发布日期’作为表索引
df_counts.rename(columns={'行数': '总内容数'}, inplace=True)
df_merge1 = pd.merge(df_counts, pivot_df, left_on='发布日期', right_on='发布日期', how='left')
df_merge2 = pd.merge(df_merge1, df_follows_views, left_on='发布日期', right_on='日期', how='left')
df_merge3 = pd.merge(df_merge2, df_cient_summary, left_on='发布日期', right_on='日期', how='left')
df_merge4 = df_merge3[::-1]
#df_merge = df_merge.drop(columns=["日期"])





#df_analysis = df_merge[['发布日期', '总内容数', '曝光人数', '店铺客户数']].copy()

In [82]:
#分析内容列表（内容id，内容分类，累计曝光，高曝光日期和值），需要结合df_follows_contents、df_follows_contents_new、df_follows_contents_list


test44 = '''
仙女必备连衣裙！浪漫又优雅，让你成为夏日焦点！🌸 你最喜欢什么颜色的连衣裙呢？

内容ID：24296002467310900009
    gobuybaby   清单  2025-04-12
95


25


趋势
'''

def extract_info2(text):


    lines = text.strip().split('\n')

    title = lines[0]
    content_id_line = lines[2].split('：')
    content_id = content_id_line[1].strip()
    info_line = lines[3].split()
    uploader = info_line[0].strip() if info_line else None
    content_type = info_line[1].strip() if len(info_line) > 1 else None
    publish_date = info_line[2].strip() if len(info_line) > 2 else None
    reach_str = lines[4].strip().replace(',', '') if len(lines) > 4 and lines[4].strip() else None
    reach = int(reach_str) if reach_str else None

    views_str = lines[6].strip().replace(',', '') if len(lines) > 6 and lines[6].strip() else None
    views = int(views_str) if views_str else None
    data = {
        "内容标题": title,
        "内容ID": content_id,
        "上传者": uploader,
        "内容类型": content_type,
        "发布日期": publish_date,
        "曝光人数": reach,
        "浏览人数": views,
    }
    return data
print(extract_info2(test44))

# 应用函数，提取所有行的信息
df_follows_contents_new['拆分'] = df_follows_contents_new["上新内容列表按曝光人数排序"].apply(extract_info2)
df_follows_contents_list['拆分'] = df_follows_contents_list["清单内容列表按曝光人数排序"].apply(extract_info2)


# 将提取到的数据转换为新的DataFrame
#df_extracted_new = pd.DataFrame(df_follows_contents_new['拆分'].tolist())
#df_follows_contents2 = extracted_df




{'内容标题': '仙女必备连衣裙！浪漫又优雅，让你成为夏日焦点！🌸 你最喜欢什么颜色的连衣裙呢？', '内容ID': '24296002467310900009', '上传者': 'gobuybaby', '内容类型': '清单', '发布日期': '2025-04-12', '曝光人数': 95, '浏览人数': None}


In [ ]:
# df_follows_contents2.to_excel('')
df_merge4.to_excel('TEST.xlsx')

## 分析店铺客户数、关注内容曝光数、关注内容发布数的关系


In [37]:
# 日期、店铺客户数、关注曝光数，关注发布数

# Ensure df_merge is defined by running the necessary cells above
df_analysis = df_merge4[['发布日期', '总内容数', '曝光人数', '店铺客户数']].copy()
df_analysis[['总内容数', '曝光人数', '店铺客户数']] = df_analysis[['总内容数', '曝光人数', '店铺客户数']].replace(',', '', regex=True).apply(pd.to_numeric, errors='coerce')
print(df_analysis.head(10))

cols_to_convert = ['总内容数', '曝光人数', '店铺客户数']
for col in cols_to_convert:
    df_analysis[col] = pd.to_numeric(df_analysis[col], errors='coerce')

df_analysis['曝光比例'] = (df_analysis['曝光人数'] / df_analysis['店铺客户数'] * 100).round(1)
df_analysis.to_excel('TEST2.xlsx')
#以店铺

           发布日期  总内容数    曝光人数     店铺客户数
324  2025-04-23     7  5981.0   98428.0
323  2025-04-22     8  5911.0   91812.0
322  2025-04-21     8  5955.0   93854.0
321  2025-04-20    28  6155.0  110233.0
320  2025-04-19    27  6908.0  101115.0
319  2025-04-18    44  6195.0  119401.0
318  2025-04-17    25  4413.0  121879.0
317  2025-04-16    22  4158.0  138786.0
316  2025-04-15    11  3923.0  109763.0
315  2025-04-14    18  5789.0  111130.0
